#### Data Generation with your own reactions, modify
#### dataframe.to_csv('datasets/{rock/paper/scissor}.csv', header=True, index=None)
#### to save the properly reactions in the folders to train the model afterward
#### also modify "new_dataframe['target'] = 2" por 0=rock, 1=paper, 2=scissor, 
#### to define the dependent variable that the model must predict 

In [9]:
import cv2
import mediapipe as mp
import csv

import numpy as np
import pandas as pd

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

dataframe = None

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        landmarks_hand = {}

        if results.multi_hand_landmarks:

            for index, landmark in enumerate(results.multi_hand_landmarks[0].landmark):
                x = landmark.x
                y = landmark.y
                z = landmark.z

                landmarks_hand[f'landmark_{index}_x'] = x
                landmarks_hand[f'landmark_{index}_y'] = y
                landmarks_hand[f'landmark_{index}_z'] = z

            new_dataframe = pd.DataFrame(landmarks_hand, index=range(len(landmarks_hand.values())))
            new_dataframe['target'] = 2

            if dataframe is not None:
                dataframe = pd.concat((dataframe, new_dataframe))
            else:
                dataframe = new_dataframe.copy()

            dataframe.to_csv('datasets/scissor.csv', header=True, index=None)

        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

rock = pd.read_csv('datasets/rock.csv')
scissor = pd.read_csv('datasets/scissor.csv')
paper = pd.read_csv('datasets/paper.csv')

original_dataframe = pd.concat((rock, paper, scissor))
original_dataframe = original_dataframe.sample(frac=1)

x = original_dataframe.drop('target', axis=1)
y = original_dataframe['target']

one_hot = OneHotEncoder(sparse_output=False)
y = one_hot.fit_transform(y.to_numpy().reshape(-1, 1))

print(y)
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.7)

test_x, ev_x, test_y, ev_y = train_test_split(x, y, train_size=0.7)

train_x

[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


,landmark_0_x,landmark_0_y,landmark_0_z,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,landmark_3_x,...,landmark_17_z,landmark_18_x,landmark_18_y,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z
1046,0.174002,0.885061,3.623993e-10,0.252236,0.882192,-0.033130,0.324448,0.811234,-0.062639,0.335721,...,-0.095119,0.238400,0.642186,-0.128928,0.236347,0.716632,-0.130315,0.228465,0.772228,-0.124228
3418,0.273122,0.822304,5.934432e-09,0.368654,0.785282,-0.020860,0.460080,0.673556,-0.038508,0.485609,...,-0.080261,0.281512,0.530850,-0.092801,0.285839,0.625360,-0.079802,0.264445,0.640064,-0.063597
2149,0.231080,0.825721,-6.797549e-09,0.301042,0.739255,-0.027353,0.325068,0.606284,-0.064363,0.299450,...,-0.142895,0.204141,0.610256,-0.171498,0.260334,0.651911,-0.171210,0.288194,0.684098,-0.167162
4829,0.349277,0.737306,5.077187e-09,0.434033,0.749160,-0.005394,0.528513,0.700791,-0.014675,0.576771,...,-0.055014,0.427535,0.500535,-0.067418,0.408746,0.574316,-0.058929,0.384806,0.597873,-0.047639
4255,0.314325,0.833562,5.931614e-09,0.404099,0.823439,-0.010623,0.501720,0.739797,-0.025153,0.543890,...,-0.077350,0.366390,0.567770,-0.090544,0.353362,0.659235,-0.076661,0.328061,0.670652,-0.060323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,0.290832,0.765791,3.530613e-09,0.250158,0.667337,0.050529,0.258757,0.588537,0.061056,0.297775,...,-0.050055,0.406976,0.593049,-0.013238,0.402408,0.619843,0.008416,0.392993,0.639904,0.016299
3779,0.258841,0.764729,1.807532e-08,0.345623,0.831599,-0.075467,0.443984,0.850205,-0.145312,0.524127,...,-0.184410,0.190018,0.494877,-0.231278,0.162840,0.420587,-0.254400,0.147519,0.349513,-0.276611
3758,0.205777,0.909890,1.320551e-08,0.185068,0.745463,0.097039,0.218611,0.653786,0.116275,0.270374,...,-0.059918,0.361978,0.682947,0.011659,0.338039,0.713788,0.067388,0.324346,0.746433,0.100854
4106,0.142824,0.753697,9.875314e-09,0.119730,0.604410,0.087031,0.144797,0.525730,0.105428,0.186695,...,-0.057377,0.291948,0.543388,-0.000571,0.272894,0.574556,0.042138,0.255652,0.600944,0.067627


In [11]:
from keras.layers import MaxPooling2D, Flatten, Dense, Dropout, Convolution2D
from keras import Sequential
import tensorflow as tf

model = Sequential()
model.add(Dense(63, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))


In [12]:
print(x.shape)

(18333, 63)


In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['Accuracy'])

In [ ]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=10)

In [15]:
predict = ev_x.iloc[0].to_numpy().reshape(1, -1)
print(model.predict(predict))
print(ev_y[0])

predict = ev_x.iloc[10].to_numpy().reshape(1, -1)
print(model.predict(predict))
print(ev_y[10])

predict = ev_x.iloc[13].to_numpy().reshape(1, -1)
print(model.predict(predict))
print(ev_y[13])

1/1 [==============================] - 0s 35ms/step
[[9.9983048e-01 1.6702335e-04 2.5497400e-06]]
[1. 0. 0.]
1/1 [==============================] - 0s 15ms/step
[[5.6292080e-05 9.3001645e-06 9.9993443e-01]]
[0. 0. 1.]
1/1 [==============================] - 0s 13ms/step
[[9.9997795e-01 1.0967216e-05 1.1121751e-05]]
[1. 0. 0.]


In [ ]:
import numpy as np

def get_machine_move():
    return np.random.choice([0,1,2])

def check_move(imagen, machine_move, prediction):
    if machine_move == 0:
        if prediction == 0:
            text = 'hemos empatado yo he sacado' + type_predict[machine_move]

            cv2.putText(imagen, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text

        elif prediction == 1:
            text = 'has ganado yo he sacado ' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text

        else:
            text = 'has perdido yo he sacado ' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text

    elif machine_move == 1:
        if prediction == 0:
            text = 'has perdido yo he sacado ' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text

        elif prediction == 1:
            text = 'hemos empatado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text
        else:
            text = 'has ganado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text

    else:
        if prediction == 0:
            text = 'has ganado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text

        elif prediction == 1:
            text = 'has perdido yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)
            return text
        else:
            text = 'hemos empatado yo he sacado' + type_predict[machine_move]

            cv2.putText(image, text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

            return text


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
type_predict = {0: 'rock', 1:'paper', 2:'scissors'}
image_predict = {0: cv2.imread("filtros/piedra.jpg"), 1:cv2.imread("filtros/paper.jpg"), 2:cv2.imread("filtros/tijeras.jpg")}
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 0, 0)
thickness = 1
wait = False
dataframe = None
count = 0
cap = cv2.VideoCapture(0)
timer = 3
wait_5_seconds = False
win_text = ''
win_image = np.array([])

with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        count += 1
        success, image = cap.read()

        image = cv2.flip(image, 1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.putText(image, str(timer), (int(image.shape[1]/2), int(image.shape[0]/2)), font, fontScale, color, thickness, cv2.LINE_AA)

        if wait_5_seconds and count < 200:
            cv2.putText(image, win_text, (0, 100), font,
                            fontScale, color, thickness, cv2.LINE_AA)

        elif count >= 200 and wait_5_seconds:
            wait_5_seconds = False
            count = 0
            timer = 3
            win_text = ''
            win_image = np.array([])

        if count >= 30 and timer > 0:
            timer-=1
            count = 0

        if timer == 0 and not wait_5_seconds:
            results = hands.process(image)

            landmarks_hand = {}

            if results.multi_hand_landmarks:

                for index, landmark in enumerate(results.multi_hand_landmarks[0].landmark):
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z

                    landmarks_hand[f'landmark_{index}_x'] = x
                    landmarks_hand[f'landmark_{index}_y'] = y
                    landmarks_hand[f'landmark_{index}_z'] = z

                machine_move = get_machine_move()

                predict = np.array(list(landmarks_hand.values())).reshape(1, -1)
                predict = np.argmax(model.predict(predict))
                print(predict)
                win_text = check_move(image, machine_move, predict)

                win_image = image_predict[machine_move]
                win_image = cv2.resize(win_image, (200, 200))
                cv2.imshow('Machine', win_image)

                count = 0
                wait_5_seconds = True

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()